# Recommend outfits based on seasons

In [64]:
import numpy as np
import pandas as pd
import pickle
import re

In [65]:
from pydrive.auth import GoogleAuth
from pydrive.drive import GoogleDrive
from google.colab import auth
from oauth2client.client import GoogleCredentials

In [66]:
auth.authenticate_user()
gauth = GoogleAuth()
gauth.credentials = GoogleCredentials.get_application_default()
drive = GoogleDrive(gauth)

#Testing 1

In [67]:
link2 = 'https://drive.google.com/file/d/1ypzPWMt5FqtUYtS0kN_OpXjtYKXZxno9/view?usp=sharing'
id2 = link2.split("/")[-2]
 
downloaded12 = drive.CreateFile({'id':id2})
downloaded12.GetContentFile('myntra-com-data.csv') 
num = pd.read_csv('myntra-com-data.csv', error_bad_lines=False)
df = num.loc[:, ['link', 'brand', 'title', 'product_type', 'images', 'product_details', 'body']]
# df.columns = ['Bust', 'Waist', 'Hip']
df.dropna(inplace=True)
df.head()

<ipython-input-67-9e00ca131a02>:6: FutureWarning: The error_bad_lines argument has been deprecated and will be removed in a future version. Use on_bad_lines in the future.


  num = pd.read_csv('myntra-com-data.csv', error_bad_lines=False)
Skipping line 20: expected 25 fields, saw 26
Skipping line 106: expected 25 fields, saw 26
Skipping line 116: expected 25 fields, saw 26
Skipping line 119: expected 25 fields, saw 26
Skipping line 132: expected 25 fields, saw 26
Skipping line 139: expected 25 fields, saw 26
Skipping line 239: expected 25 fields, saw 26
Skipping line 284: expected 25 fields, saw 26
Skipping line 294: expected 25 fields, saw 26
Skipping line 332: expected 25 fields, saw 26
Skipping line 389: expected 25 fields, saw 26
Skipping line 434: expected 25 fields, saw 26
Skipping line 438: expected 25 fields, saw 26
Skipping line 439: expected 25 fields, saw 26
Skipping line 453: expected 25 fields, saw 26
Skipping line 491: expected 25 fields, saw 26
Skipping line 507: expect

,link,brand,title,product_type,images,product_details,body
0,https://www.myntra.com/tops/imara/imara-women-...,IMARA,IMARA Women Black Solid Top,Top,http://assets.myntassets.com/v1/assets/images/...,"Black solid woven regular top,has a V-neck, th...","Black solid woven regular top,has a V-neck, th..."
1,https://www.myntra.com/kurtas/house-of-pataudi...,House of Pataudi,House of Pataudi Men Black Printed Straight Kurta,Straight Kurta,http://assets.myntassets.com/v1/assets/images/...,"Black printed straight kurta, has a mandarin c...","Black printed straight kurta, has a mandarin c..."
3,https://www.myntra.com/dupatta/dupatta-bazaar/...,Dupatta Bazaar,Dupatta Bazaar Pink Dupatta,Dupatta,http://assets.myntassets.com/v1/assets/images/...,Pink dupatta&nbsp;with crinkled effect,Pink dupatta&nbsp;with crinkled effect\nLength...
4,https://www.myntra.com/kurta-sets/manyavar/man...,Manyavar,Manyavar Men Yellow & White Self Design Kurta ...,Kurta with Churidar,http://assets.myntassets.com/v1/assets/images/...,Yellow and white self design kurta with churid...,Yellow and white self design kurta with churid...
5,https://www.myntra.com/kurta-sets/wintage/wint...,Wintage,Wintage Men Beige Solid Kurta with Pyjamas,Kurta with Pyjamas,http://assets.myntassets.com/v1/assets/images/...,Beige solid kurta with pyjamas Beige straight ...,Beige solid kurta with pyjamas Beige straight ...


In [68]:
len(df.index)

12873

In [69]:
cols = ['body']
df_subset = df.loc[:, cols]
print(df_subset)

                                                    body
0      Black solid woven regular top,has a V-neck, th...
1      Black printed straight kurta, has a mandarin c...
3      Pink dupatta&nbsp;with crinkled effect\nLength...
4      Yellow and white self design kurta with churid...
5      Beige solid kurta with pyjamas Beige straight ...
...                                                  ...
14226  Grey and red floral print woven maxi dress wit...
14227  Red and white self design kurta with pyjamas R...
14228  Green and yellow printed kurta with pyjamas  G...
14229  Green printed cardigan, has a V-neck, long sle...
14230  Green and white printed straight kurta, has a ...

[12873 rows x 1 columns]


In [70]:
df1 = df
df1['detail_desc'] = df['body'].fillna('') + ' ' + df['product_details'].fillna('')
df1.head()

,link,brand,title,product_type,images,product_details,body,detail_desc
0,https://www.myntra.com/tops/imara/imara-women-...,IMARA,IMARA Women Black Solid Top,Top,http://assets.myntassets.com/v1/assets/images/...,"Black solid woven regular top,has a V-neck, th...","Black solid woven regular top,has a V-neck, th...","Black solid woven regular top,has a V-neck, th..."
1,https://www.myntra.com/kurtas/house-of-pataudi...,House of Pataudi,House of Pataudi Men Black Printed Straight Kurta,Straight Kurta,http://assets.myntassets.com/v1/assets/images/...,"Black printed straight kurta, has a mandarin c...","Black printed straight kurta, has a mandarin c...","Black printed straight kurta, has a mandarin c..."
3,https://www.myntra.com/dupatta/dupatta-bazaar/...,Dupatta Bazaar,Dupatta Bazaar Pink Dupatta,Dupatta,http://assets.myntassets.com/v1/assets/images/...,Pink dupatta&nbsp;with crinkled effect,Pink dupatta&nbsp;with crinkled effect\nLength...,Pink dupatta&nbsp;with crinkled effect\nLength...
4,https://www.myntra.com/kurta-sets/manyavar/man...,Manyavar,Manyavar Men Yellow & White Self Design Kurta ...,Kurta with Churidar,http://assets.myntassets.com/v1/assets/images/...,Yellow and white self design kurta with churid...,Yellow and white self design kurta with churid...,Yellow and white self design kurta with churid...
5,https://www.myntra.com/kurta-sets/wintage/wint...,Wintage,Wintage Men Beige Solid Kurta with Pyjamas,Kurta with Pyjamas,http://assets.myntassets.com/v1/assets/images/...,Beige solid kurta with pyjamas Beige straight ...,Beige solid kurta with pyjamas Beige straight ...,Beige solid kurta with pyjamas Beige straight ...


In [71]:
import pandas as pd
import re

# Define the keywords and their associated seasons
keywords = {
    "Summer": ["summer", "cotton", "shorts", "graphic tees", "sneakers", "mini skirts", "sleeveless", "capris"],
    "Winter": ["wollen", "blazers", "blouse", "classy", "sophisticated", "winter", "silk", "long sleeves"],
    "Spring": ["Off-shoulder", "tank top", "midi dresses", "floral", "jeans", "jumpsuit", "three-quarter sleeves", "long sleeves"],
    "Fall": ["Turtlenecks", "Sweaters", "Denim", "Midi skirts", "wrap dress", "Maxi dresses"]
}

# Define a function to match keywords to a given text
def match_keywords(text):
    for season, keys in keywords.items():
        for key in keys:
            # Use regular expression to find the keyword in the text column
            pattern = f'\\b{key}\\b'
            match = re.search(pattern, text, re.IGNORECASE)
            if match:
                return season
    return 'All Season'

# Apply the match_keywords function to the text column of the dataset
df1['season'] = df['detail_desc'].apply(match_keywords)

df['season'] = df['season'].astype(str)


# View the resulting dataframe
print(df1.head(50))

                                                 link                 brand  \
0   https://www.myntra.com/tops/imara/imara-women-...                 IMARA   
1   https://www.myntra.com/kurtas/house-of-pataudi...      House of Pataudi   
3   https://www.myntra.com/dupatta/dupatta-bazaar/...        Dupatta Bazaar   
4   https://www.myntra.com/kurta-sets/manyavar/man...              Manyavar   
5   https://www.myntra.com/kurta-sets/wintage/wint...               Wintage   
7   https://www.myntra.com/kurtas/the-indian-garag...  The Indian Garage Co   
8   https://www.myntra.com/kurtas/anouk/anouk-wome...                 Anouk   
9   https://www.myntra.com/kurta-sets/salwar-studi...         SALWAR STUDIO   
10  https://www.myntra.com/palazzos/clora-creation...        Clora Creation   
11  https://www.myntra.com/kurtas/libas/libas-wome...                 Libas   
12  https://www.myntra.com/kurta-sets/libas/libas-...                 Libas   
13  https://www.myntra.com/kurtas/anouk/anouk-wome..

In [72]:
print(df1['season'])

0            Spring
1            Winter
3        All Season
4            Winter
5            Summer
            ...    
14226        Spring
14227        Summer
14228        Summer
14229        Winter
14230    All Season
Name: season, Length: 12873, dtype: object


In [73]:
# Convert the DataFrame to JSON format
json_data = df1.to_json(orient='records')

# Convert the JSON data back to a DataFrame
df_from_json = pd.read_json(json_data)

# Save the DataFrame to a CSV file
df_from_json.to_csv('season.csv', index=False)

print(json_data)

IOPub data rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_data_rate_limit`.

Current values:
NotebookApp.iopub_data_rate_limit=1000000.0 (bytes/sec)
NotebookApp.rate_limit_window=3.0 (secs)



In [74]:
# group the outfits by season and display them
for season, group in df1.groupby('season'):
    print(f'Outfits for {season}:')
    print(group)

Outfits for All Season:
                                                    link             brand  \
3      https://www.myntra.com/dupatta/dupatta-bazaar/...    Dupatta Bazaar   
10     https://www.myntra.com/palazzos/clora-creation...    Clora Creation   
41     https://www.myntra.com/leggings/global-desi/gl...       Global Desi   
51     https://www.myntra.com/palazzos/abhishti/abhis...          Abhishti   
54     https://www.myntra.com/kurtas/libas/libas-wome...             Libas   
...                                                  ...               ...   
14173  https://www.myntra.com/trousers/kaanchie-nangg...  KAANCHIE NANGGIA   
14199  https://www.myntra.com/kurtas/libas/libas-wome...             Libas   
14219                     https://www.myntra.com/8748315         Ethnicity   
14224                     https://www.myntra.com/9783983    Clora Creation   
14230                     https://www.myntra.com/2012151             Anouk   

                                       

In [75]:
import pandas as pd

def recommend_outfits(season):
    # filter the outfits by season
    season_outfits = df1[df1['season'] == season]

    # recommend the outfits
    if season_outfits.empty:
        print(f'Sorry, we don\'t have any outfits for {season} season.')
    else:
        print(f'Recommended outfits for {season} season:')
        print(season_outfits[['title', 'link', 'images', 'body']])

In [76]:
recommend_outfits('Summer')

Recommended outfits for Summer season:
                                                   title  \
5             Wintage Men Beige Solid Kurta with Pyjamas   
7      The Indian Garage Co Men Blue & White Printed ...   
9      SALWAR STUDIO Boys Orange & White Printed Kurt...   
14                     Global Desi White Net Sheer Shrug   
16     612 league Boys Red & Beige Solid Kurta with C...   
...                                                  ...   
14220  Anouk Women Multicoloured Checked Mangalgiri A...   
14221           Libas Women Green Printed Straight Kurta   
14225                             Biba Red Printed Kurti   
14227  Freehand Men Red & White Self-Design Kurta wit...   
14228  LilPicks Boys Green & Yellow Printed Kurta wit...   

                                                    link  \
5      https://www.myntra.com/kurta-sets/wintage/wint...   
7      https://www.myntra.com/kurtas/the-indian-garag...   
9      https://www.myntra.com/kurta-sets/salwar-studi...   


In [77]:
recommend_outfits('Winter')

Recommended outfits for Winter season:
                                                   title  \
1      House of Pataudi Men Black Printed Straight Kurta   
4      Manyavar Men Yellow & White Self Design Kurta ...   
8                  Anouk Women Maroon Solid A-Line Kurta   
11       Libas Women Blue & White Striped Straight Kurta   
12     Libas Women Coffee Brown Solid Kurta with Pala...   
...                                                  ...   
14167                              Fabindia Maroon Kurti   
14185  DEYANN Men Blue & Cream-Coloured Solid Kurta w...   
14192  Khoday Williams Men Burgundy Woven Design Stra...   
14201  Manyavar Men Blue & White Self-Design Kurta wi...   
14229     Anouk Women Green Floral Foil Printed Cardigan   

                                                    link  \
1      https://www.myntra.com/kurtas/house-of-pataudi...   
4      https://www.myntra.com/kurta-sets/manyavar/man...   
8      https://www.myntra.com/kurtas/anouk/anouk-wome...   


In [78]:
recommend_outfits('Spring')

Recommended outfits for Spring season:
                                                   title  \
0                            IMARA Women Black Solid Top   
13               Anouk Women Navy Printed Straight Kurta   
24     Sangria Women Cream-Coloured Solid Straight Kurta   
37             Sangria Women Maroon Printed A-Line Kurta   
39     Shree Women Yellow & Purple Printed Straight K...   
...                                                  ...   
14202    Global Desi Women Yellow Printed Straight Kurta   
14207    Global Desi Women Pink Printed Culotte Jumpsuit   
14222      Anouk Women White & Blue Printed A-Line Kurta   
14223  Rangriti Women Pink & Orange Printed Anarkali ...   
14226     Shree Women Grey & Red Floral Print Maxi Dress   

                                                    link  \
0      https://www.myntra.com/tops/imara/imara-women-...   
13     https://www.myntra.com/kurtas/anouk/anouk-wome...   
24     https://www.myntra.com/kurtas/sangria/sangria-...   


In [79]:
recommend_outfits('Fall')

Recommended outfits for Fall season:
                                                   title  \
836    MABISH by Sonal Jain Women Orange Solid Wrap D...   
2426   RARE ROOTS Women Mustard Yellow & Maroon Polka...   
5784            AKS Women Navy Blue Dyed Maxi Wrap Dress   
9393                   Rangriti Women Assorted Solid Top   
10180                   INDYA Women Grey Embellished Top   

                                                    link  \
836    https://www.myntra.com/dresses/mabish-by-sonal...   
2426   https://www.myntra.com/dresses/rare-roots/rare...   
5784   https://www.myntra.com/dresses/aks/aks-women-n...   
9393   https://www.myntra.com/tops/rangriti/rangriti-...   
10180                     https://www.myntra.com/7445751   

                                                  images  \
836    http://assets.myntassets.com/v1/assets/images/...   
2426   http://assets.myntassets.com/v1/assets/images/...   
5784   http://assets.myntassets.com/v1/assets/images/...   
9

In [81]:
recommend_outfits('All Season')

Recommended outfits for All Season season:
                                                   title  \
3                            Dupatta Bazaar Pink Dupatta   
10     Clora Creation Women Navy Blue Wide Leg Printe...   
41                            Global Desi Blue Treggings   
51     Abhishti Women Sea Green & White Solid Flared ...   
54          Libas Women Green Yoke Design Straight Kurta   
...                                                  ...   
14173    KAANCHIE NANGGIA Women Yellow Solid Dhoti Pants   
14199           Libas Women Brown Checked Straight Kurta   
14219       Ethnicity Women Pink Solid Straight Palazzos   
14224  Clora Creation Women Black & Gold-Coloured Pri...   
14230   Anouk Women Green & White Printed Straight Kurta   

                                                    link  \
3      https://www.myntra.com/dupatta/dupatta-bazaar/...   
10     https://www.myntra.com/palazzos/clora-creation...   
41     https://www.myntra.com/leggings/global-desi/gl...